In [1]:
# import
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.metrics import r2_score

In [2]:
finalists_clean_df = pd.read_csv('Data/finalists_cleaned.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/finalists_cleaned.csv'

In [ ]:
# Display the first few rows to verify
print(finalists_clean_df.head())

In [ ]:
finalists_clean_df.shape

In [ ]:
finalists_clean_df.info()

In [ ]:
y = finalists_clean_df['final_place']

# Features: drop target and categorical variables
X = finalists_clean_df.drop(columns=['final_place', 'country', 'style'])

# Define numeric feature names (all remaining)
numeric_features = X.columns.tolist()


In [ ]:
y.head()

In [ ]:
# Split the dataset (74/26)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.26, random_state=42)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
model = Pipeline(steps=[
    ('imputer', SimpleImputer(fill_value=None)),
    ('regressor', LinearRegression())
])

In [ ]:
# Fit and predict
model.fit(X_train, y_train)


In [ ]:
reg = model.named_steps['regressor']


In [ ]:
print("Intercept:", reg.intercept_)
print("Coefficients:", reg.coef_)

feature_names = X_train.columns
coef_table = list(zip(feature_names, reg.coef_))
for name, coef in coef_table:
    print(f"{name:30} {coef:>10.4f}")


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
import sys
import os

# Use the current working directory instead of __file__
current_directory = os.getcwd()

# Add the path to the 'Modules' folder (adjust the path to reach the Modules folder)
sys.path.append(os.path.join(current_directory, 'Modules'))

# Now import the evalute function
from machine_learning.evaluate import evaluate_model

# Assuming you have your model, X_test, and y_test defined somewhere
evaluation_results = evaluate_model(model, X_test, y_test)

# Print the evaluation metrics in a formatted way
print("Model Evaluation Results:")
print("----------------------------")
for metric, value in evaluation_results.items():
    if metric != 'predictions':
        print(f"{metric}: {value:.4f}")
    else:
        print(f"\nPredictions (showing first 10 for brevity):")
        print(", ".join([f"{v:.2f}" for v in value[:10]]))  # Only show the first 10 predictions

In [ ]:
y_pred

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # Diagonal
plt.xlabel('Actual Final Place')
plt.ylabel('Predicted Final Place')
plt.title('Predicted vs Actual Final Place')
plt.grid(True)
plt.show()

In [ ]:
y_test.iloc[0]

In [ ]:
y_pred[0]

## Store model

In [ ]:
import joblib

In [ ]:
# Store the model in a file
model_file = 'Models/finalistfit.pkl'

In [ ]:
# save the model
joblib.dump(model, model_file)